In [187]:
from dateutil.parser import parse
from datetime import datetime, timedelta
import pytz

def infer_meeting_datetime(email_content, request_datetime_str, duration_mins=30):
    """
    Parse email content to extract meeting datetime and return formatted timestamps
    
    Args:
        email_content: The email text containing time references
        request_datetime_str: Reference datetime string (e.g., "19-07-2025T12:34:55")
        duration_mins: Meeting duration in minutes
    
    Returns:
        dict: {
            'start_timestamp': formatted start time string,
            'end_timestamp': formatted end time string,
            'start_datetime': datetime object,
            'end_datetime': datetime object
        }
    """
    
    # Parse the reference datetime
    reference_date = parse(request_datetime_str)
    
    # Add timezone if not present (assuming IST based on your format)
    if reference_date.tzinfo is None:
        ist = pytz.timezone('Asia/Kolkata')
        reference_date = ist.localize(reference_date)
    
    # For now, using your LLM-extracted time string
    # In practice, this would come from your LLM call
    meeting_time_string = "next Thursday at 2pm"  # This comes from LLM
    
    try:
        # Parse the meeting time relative to reference date
        inferred_datetime = parse(meeting_time_string, default=reference_date)
        
        # Ensure timezone consistency
        if inferred_datetime.tzinfo is None:
            inferred_datetime = reference_date.tzinfo.localize(inferred_datetime.replace(tzinfo=None))
        
        # Calculate end time
        end_datetime = inferred_datetime + timedelta(minutes=duration_mins)
        
        # Format as required timestamps
        start_timestamp = inferred_datetime.strftime("%Y-%m-%dT%H:%M:%S%z")
        end_timestamp = end_datetime.strftime("%Y-%m-%dT%H:%M:%S%z")
        
        # Format timezone properly (add colon)
        start_timestamp = start_timestamp[:-2] + ':' + start_timestamp[-2:]
        end_timestamp = end_timestamp[:-2] + ':' + end_timestamp[-2:]
        
        return {
            'start_timestamp': start_timestamp,
            'end_timestamp': end_timestamp,
            'start_datetime': inferred_datetime,
            'end_datetime': end_datetime
        }
        
    except Exception as e:
        print(f"Error parsing datetime: {e}")
        return None

def extract_meeting_info_with_llm(email_content, request_datetime_str):
    """
    Complete function that would integrate with your LLM to extract time and duration
    """
    # Your LLM call would go here to extract:
    # 1. Time constraint (e.g., "Thursday at 2pm")
    # 2. Duration (e.g., 30 minutes)
    
    # Mock LLM response for demonstration
    llm_response = {
        "time_constraint": "Thursday at 2pm",
        "duration_minutes": 30
    }
    
    # Use the extracted info to get timestamps
    reference_date = parse(request_datetime_str)
    
    try:
        # Parse the time constraint
        meeting_datetime = parse(llm_response["time_constraint"], default=reference_date)
        
        # Add IST timezone if missing
        if meeting_datetime.tzinfo is None:
            ist = pytz.timezone('Asia/Kolkata')
            meeting_datetime = ist.localize(meeting_datetime)
        
        # Calculate end time
        duration = llm_response["duration_minutes"]
        end_datetime = meeting_datetime + timedelta(minutes=duration)
        
        # Format timestamps for output JSON
        start_timestamp = meeting_datetime.strftime("%Y-%m-%dT%H:%M:%S%z")
        end_timestamp = end_datetime.strftime("%Y-%m-%dT%H:%M:%S%z")
        
        # Add colon to timezone format (+05:30 instead of +0530)
        start_timestamp = start_timestamp[:-2] + ':' + start_timestamp[-2:]
        end_timestamp = end_timestamp[:-2] + ':' + end_timestamp[-2:]
        
        return {
            "EventStart": start_timestamp,
            "EventEnd": end_timestamp,
            "Duration_mins": str(duration)
        }
        
    except Exception as e:
        print(f"Error processing meeting info: {e}")
        return None

# Example Usage matching your input format:
request_datetime = '19-07-2025T12:34:55'  # From input JSON
email_content = "Hi team, let's meet on Thursday for 30 minutes to discuss the status of Agentic AI Project."

result = extract_meeting_info_with_llm(email_content, request_datetime)
print(result)
# Output: {
#     "EventStart": "2025-07-24T14:00:00+05:30",
#     "EventEnd": "2025-07-24T14:30:00+05:30", 
#     "Duration_mins": "30"
# }

{'EventStart': '2025-07-24T14:34:55+05:30', 'EventEnd': '2025-07-24T15:04:55+05:30', 'Duration_mins': '30'}


In [191]:
reference_date = parse("19-07-2025T12:34:55")  # Saturday, July 19, 2025

# Parse relative to reference date
meeting_time = parse("Monday at 2 am", default=reference_date)
# This interprets "Thursday" as the NEXT Thursday after July 19
# Result: July 24, 2025 at 2:00 PM
meeting_time

datetime.datetime(2025, 7, 21, 2, 34, 55)

In [192]:
parse("Thursday at 2pm", default=reference_date)     # Next Thursday at 2 PM
parse("tomorrow at 3pm", default=reference_date)     # Tomorrow at 3 PM
parse("next week", default=reference_date)           # Next week (same day/time)
parse("Friday", default=reference_date)              # Next Friday

ParserError: Unknown string format: tomorrow at 3pm

In [ ]:
def extract_meeting_time_hybrid(email_content, reference_date_str):
    """
    Use LLM to extract clean time constraint, then parse with enhanced logic
    """
    # Step 1: LLM extracts clean time constraint
    # Your existing LLM call that returns something like:
    # {"time_constraint": "Thursday at 2pm", "duration": 30}
    
    # Step 2: Parse with fallback logic
    reference_date = parse(reference_date_str)
    
    # Mock LLM response
    time_constraint = "Thursday at 2pm"  # From your LLM
    
    # Try enhanced parsing first, fall back to dateutil
    meeting_time = parse_relative_date(time_constraint, reference_date)
    
    if meeting_time is None:
        try:
            meeting_time = parse(time_constraint, default=reference_date)
        except:
            # Default to next day at 2pm if all else fails
            meeting_time = reference_date + timedelta(days=1)
            meeting_time = meeting_time.replace(hour=14, minute=0, second=0)
    
    return meeting_time